In [1]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-26aoo7io
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-26aoo7io
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=a9aaf2e97725d13b92c11f803e8ef9b999a3cca85c0420e75155c3c71003c041
  Stored in directory: /tmp/pip-ephem-wheel-cache-s3m1n33p/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
!/usr/local/cuda/bin/nvcc --version
#!pip install --upgrade git+https://github.com/frehseg/nvcc4jupyter.git
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cuda-install-samples-11.2.sh ~ && cd /root/NVIDIA_CUDA-11.2_Samples/

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xnpmu7q9
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xnpmu7q9
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
/bin/bash: line 1: cuda-install-samples-11.2.sh: command not found


In [3]:
%%cuda --name my_curand.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cstdlib>
#include <malloc.h>
#include <curand.h>
void mult_cpu(float* A, float* B, float* C, int nA, int nB) {
    for (int i = 0; i < nA; ++i) {
        for (int j = 0; j < nB; ++j) {
            for (int k = 0; k < nB; ++k)
                C[i * nB + j] += A[i * nB + k] * B[k * nB + j];
        }
    }
}
// GPU_fill_rand() - Функция случайной генерации матрицы
// Заполнить массив A(nr_rows_A, nr_cols_A) случайными числами на графическом процессоре
void GPU_fill_rand(float* A, int nr_rows_A, int nr_cols_A) {
    // Создание генератора псевдо-рандомных чисел
    curandGenerator_t prng;
    curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);

    // Установка сида для генератора
    curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long) clock());

    // Заполнение матрицы случайными числами
    curandGenerateUniform(prng, A, nr_rows_A * nr_cols_A);
}

// gpu_blas_mmul() - Функция умножения матриц
// Перемножение матриц А и В с сохранением результата в матрице С
// C(m,n) = A(m,k) * B(k,n)
void gpu_blas_mmul(const float* A, const float* B, float* C, const int m, const int k, const int n) {
    int lda = m, ldb = k, ldc = m;
    const float alf = 1;
    const float bet = 0;
    const float* alpha = &alf;
    const float* beta = &bet;
    // Create a handle for CUBLAS
    cublasHandle_t handle;
    cublasCreate(&handle);
    // Do the actual multiplication
    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, alpha, A, lda, B, ldb, beta, C, ldc);
    // Destroy the handle
    cublasDestroy(handle);
}

//print_matrix() - Функция вывода матрицы
void print_matrix(float* matrix, int rows, int cols) {

    printf("%f\t*\t*\t*\t%f\n", matrix[0], matrix[rows * (rows - 1)]);
    printf("*\t*\t*\t*\t*\t*\n");
    printf("%f\t*\t*\t*\t%f\n", matrix[rows], matrix[cols * rows - 1]);
    printf("\n");

}

int main() {
    // Выделяем 3 массива на хосте
    int nr_rows_A, nr_cols_A, nr_rows_B, nr_cols_B, nr_rows_C, nr_cols_C;
    cudaEvent_t start, end;
    float time = 0.0f;
    cudaEventCreate(&start);
    cudaEventCreate(&end);

    // Используются квадратные матрицы
    nr_rows_A = nr_cols_A = nr_rows_B = nr_cols_B = nr_rows_C = nr_cols_C = 2000;

    printf("float\nn = %d\n\n", nr_rows_A);

    float* h_A = (float*)malloc(nr_rows_A * nr_cols_A * sizeof(float));
    float* h_B = (float*)malloc(nr_rows_B * nr_cols_B * sizeof(float));
    float* h_C = (float*)malloc(nr_rows_C * nr_cols_C * sizeof(float));

    // Выделяем 3 массива на девайсе
    float* d_A, * d_B, * d_C;
    cudaMalloc(&d_A, nr_rows_A * nr_cols_A * sizeof(float));
    cudaMalloc(&d_B, nr_rows_B * nr_cols_B * sizeof(float));
    cudaMalloc(&d_C, nr_rows_C * nr_cols_C * sizeof(float));

    // Заполняем матрицы А и В случайными числами
    GPU_fill_rand(d_A, nr_rows_A, nr_cols_A);
    GPU_fill_rand(d_B, nr_rows_B, nr_cols_B);

    // Можно скопировать их на хост, чтобы вывести их
    cudaMemcpy(h_A, d_A, nr_rows_A * nr_cols_A * sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_B, d_B, nr_rows_B * nr_cols_B * sizeof(float), cudaMemcpyDeviceToHost);
    printf("A =\n");
    print_matrix(h_A, nr_rows_A, nr_cols_A);
    printf("B =\n");
    print_matrix(h_B, nr_rows_B, nr_cols_B);


    cudaEventRecord(start, 0);
    // Умножаем матрицы А и В на GPU
    gpu_blas_mmul(d_A, d_B, d_C, nr_rows_A, nr_cols_A, nr_cols_B);
    cudaEventRecord(end, 0);
    // Копируем на хост и выводим результат умножения матриц
    cudaMemcpy(h_C, d_C, nr_rows_C * nr_cols_C * sizeof(float), cudaMemcpyDeviceToHost);
    printf("C =\n");
    print_matrix(h_C, nr_rows_C, nr_cols_C);

    cudaEventElapsedTime(&time, start, end);
    printf("GPU Total time: %f\n", time / 1000);
    unsigned int startcpu =  clock(); // начальное время cpu

    mult_cpu(h_A, h_B,  h_C, nr_rows_A, nr_rows_B);

    unsigned int endcpu = clock(); // конечное время  cpu
    unsigned int timecpu = endcpu - startcpu; // искомое время  cpu
    printf("СPU mmul\nTotal time: %f\n", (double)timecpu / 1000000);
    double acceleration=timecpu/time;
    printf("Acceleration: %f\n", acceleration);
    // Освобождаем память на девайсе
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    // Освобождаем память на хосте
    free(h_A);
    free(h_B);
    free(h_C);
    return 0;
}


'File written in /content/src/my_curand.cu'

In [4]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand -lcublas

In [5]:
!/content/src/my_curand

float
n = 2000

A =
0.474572	*	*	*	0.546570
*	*	*	*	*	*
0.344717	*	*	*	0.619972

B =
0.359875	*	*	*	0.670641
*	*	*	*	*	*
0.875423	*	*	*	0.461416

C =
497.861237	*	*	*	489.197815
*	*	*	*	*	*
490.621399	*	*	*	487.822662

GPU Total time: 3.559670
СPU mmul
Total time: 93.804479
Acceleration: 26352.017578
